# **Ejercicio 7: Bases de Datos Vectoriales**

## **Nombre:** Nelson Casa
## Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset _Wikipedia Text Corpus for NLP and LLM Projects_

El corpus está disponible desde este [link](https://www.kaggle.com/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects?utm_source=chatgpt.com)

### Actividad

1. Carga el corpus


In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [2]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

Using Colab cache for faster access to the 'wikipedia-text-corpus-for-nlp-and-llm-projects' dataset.


,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


## Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este [link](https://huggingface.co/intfloat/e5-base-v2)

### Actividad

1. Normalizar el corpus
2. Definir una función `chunk_text`, y dividir los textos en _chunks_.
3. Generar embeddings por cada _chunk_

Importante: este corpus puede ser grande. Para que no se demore horas, usaremos un **subset** controlado.

In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [4]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

(   doc_id  chunk_id                                               text
 0       0         0  Anovo Anovo (formerly A Novo) is a computer se...
 1       1         0  Battery indicator A battery indicator (also kn...
 2       1         1  ad battery when in reality it indicates a prob...
 3       1         2  s that an internal standby battery needs repla...
 4       1         3  increase; in many cases the EMF remains more o...,
 79104)

## Parte 1: Embeddings en GPU (rápido)

Ahora que CUDA está disponible, cargamos el modelo en GPU y generamos embeddings
usando un subset (para no tardar horas).


In [5]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME, device="cuda")

print("Modelo en:", model.device)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Modelo en: cuda:0


### Subset de pasajes (control de tiempo)

Reducimos el número de pasajes para que el taller sea ejecutable rápido.


In [6]:
MAX_PASSAGES = 5000

passages = passages[:MAX_PASSAGES]
chunks_df = chunks_df.head(MAX_PASSAGES).reset_index(drop=True)

print("Pasajes usados:", len(passages))
print("Chunks usados:", len(chunks_df))


Pasajes usados: 5000
Chunks usados: 5000


### Generar embeddings (después del subset)

Ahora sí creamos la variable `embeddings` usando los 5000 pasajes.

In [7]:
embeddings = model.encode(
    passages,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

print(" Embeddings generados:", embeddings.shape, embeddings.dtype)


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

 Embeddings generados: (5000, 768) float32


In [8]:
print(embeddings.shape, embeddings.dtype)

(5000, 768) float32


### Embedding de una query

In [9]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

## Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este [link](https://faiss.ai/index.html)

### Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una _query_

### Instalar FAISS en Google Colab

Si aparece `ModuleNotFoundError: No module named 'faiss'`, instalamos FAISS.
En Colab normalmente funciona con `faiss-cpu`.


In [10]:
!pip -q install faiss-cpu

### 2.1 Crear el índice en FAISS (similitud coseno)

Como generamos embeddings con `normalize_embeddings=True`, podemos usar:
- `IndexFlatIP` (Inner Product) ≈ similitud coseno cuando los vectores están normalizados.


In [11]:
import faiss
import numpy as np

# Dimensión del embedding (por ejemplo 768 para e5-base-v2)
d = embeddings.shape[1]

# Índice para coseno (con embeddings normalizados)
index = faiss.IndexFlatIP(d)

print("Índice FAISS creado. Dimensión:", d)


Índice FAISS creado. Dimensión: 768


### 2.2 Cargar embeddings al índice

Aquí añadimos todos los vectores al índice FAISS.


In [12]:
# Asegurar float32 (FAISS lo requiere)
embeddings_f32 = embeddings.astype("float32")

index.add(embeddings_f32)

print("Embeddings cargados al índice.")
print("Total vectores en el índice:", index.ntotal)


Embeddings cargados al índice.
Total vectores en el índice: 5000


### 2.3 Búsqueda por similitud con una query

1) Generamos el embedding de la query usando el prefijo `query:` (E5)  
2) Buscamos Top-k resultados en FAISS  
3) Mostramos los chunks más parecidos con su score

In [13]:
import pandas as pd

def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"  # consulta
q_emb = embed_query(query_text)

k = 10
scores, idxs = index.search(q_emb, k)

print("Query:", query_text)
print("Top-k:", k)

Query: Battery measuring
Top-k: 10


## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de **metadata** y filtros.

### Qué debes implementar
1. Levantar / conectar con una instancia de Qdrant.
2. Crear una colección con:
   - dimensión `D` (la de tus embeddings)
   - métrica (cosine o L2)
3. Insertar:
   - `id`
   - `embedding`
   - `payload` (metadata: texto, título, etiquetas, etc.)
4. Consultar Top-k por similitud:
   - `query_embedding`
   - `k`

### Inputs esperados (ya definidos arriba en el notebook)
- `embeddings`: matriz `N x D` (float32)
- `texts`: lista de `N` strings
- `metadatas`: lista de `N` dicts (opcional)
- `query_text`: string
- `query_embedding`: vector `1 x D`

### Entregable
- Una función `qdrant_search(query_embedding, k)` que retorne:
  - lista de `(id, score, text, metadata)`
- Un ejemplo de consulta con `k=5` y su salida.

### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?
- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?


### Instalación de Qdrant (modo local / in-memory)

Usaremos Qdrant en modo **in-memory**, ideal para notebooks y talleres.

In [14]:
!pip -q install qdrant-client

### Conectar a Qdrant (instancia en memoria)

In [15]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# Cliente Qdrant en memoria (no necesita servidor)
client = QdrantClient(":memory:")

print("Qdrant conectado (in-memory)")

Qdrant conectado (in-memory)


### Crear colección en Qdrant

- Dimensión: D (dimensión del embedding)
- Métrica: Cosine (porque usamos embeddings normalizados)

In [16]:
D = embeddings.shape[1]
COLLECTION_NAME = "documents"

client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(
        size=D,
        distance=Distance.COSINE
    )
)

print("Colección creada:", COLLECTION_NAME)

Colección creada: documents


### Preparar datos: ids, textos y metadata

Cada punto tendrá:
- id
- vector (embedding)
- payload (texto + metadata)

In [17]:
texts = chunks_df["text"].tolist()

metadatas = [
    {
        "doc_id": int(row["doc_id"]),
        "chunk_id": int(row["chunk_id"])
    }
    for _, row in chunks_df.iterrows()
]

ids = list(range(len(embeddings)))

print("Datos preparados:")
print("IDs:", len(ids))
print("Texts:", len(texts))
print("Metadatas:", len(metadatas))

Datos preparados:
IDs: 5000
Texts: 5000
Metadatas: 5000


### Insertar embeddings y metadata en Qdrant
Usaremos `PointStruct`, que es el formato esperado por `qdrant-client`.
También insertamos en lotes (batch) para que sea rápido.

In [18]:
from qdrant_client.models import PointStruct
from tqdm.auto import tqdm

BATCH_SIZE = 512

points = []
for i in range(len(embeddings)):
    points.append(
        PointStruct(
            id=int(i),
            vector=embeddings[i].tolist(),  # Qdrant acepta list
            payload={
                "text": texts[i],
                **metadatas[i]  # doc_id, chunk_id, etc.
            }
        )
    )

# Insertar en batches
for start in tqdm(range(0, len(points), BATCH_SIZE)):
    client.upsert(
        collection_name=COLLECTION_NAME,
        points=points[start:start + BATCH_SIZE]
    )

print("Embeddings insertados en Qdrant:", len(points))


  0%|          | 0/10 [00:00<?, ?it/s]

Embeddings insertados en Qdrant: 5000


## Función entregable: qdrant_search(query_embedding, k)

Retorna:
- id
- score
- text
- metadata

In [19]:
def qdrant_search(query_embedding, k=5):
    res = client.query_points(
        collection_name=COLLECTION_NAME,
        query=query_embedding[0].tolist(),
        limit=k,
        with_payload=True,
        with_vectors=False
    )

    out = []
    for p in res.points:
        payload = p.payload or {}
        text = payload.get("text", "")
        metadata = {kk: vv for kk, vv in payload.items() if kk != "text"}
        out.append((p.id, p.score, text, metadata))
    return out

### Ejemplo de consulta con k = 5

In [20]:
query_text = "Battery measuring"
query_embedding = embed_query(query_text)

qdrant_search(query_embedding, k=5)

[(1,
  0.8618003594654431,
  "Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It is particularly important in the case of a battery electric vehicle. Some automobiles are fitted with a battery condition meter to monitor the starter battery. This meter is, essentially, a voltmeter but it may also be marked with coloured zones for easy visualization. Many newer cars no longer offer voltmeters or ammeters; instead, these vehicles typically have a light with the outline of an automotive battery on it. This can be somewhat misleading as it may be confused for an indicator of a bad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively,",
  {'doc_id': 1, 'chunk_id': 0}),
 (5,
  0.8316220322318095,
  'otective diodes cannot be used, a battery will simply destroy the diodes and damage itself. An ESR meter

### Respuestas – Parte 3: Qdrant

#### 1. ¿La métrica usada fue cosine o L2? ¿Por qué?

Se utilizó **cosine similarity** como métrica de distancia.

Esto se debe a que los embeddings fueron generados utilizando
`normalize_embeddings=True`, lo que normaliza todos los vectores a norma 1.
En este contexto, la similitud coseno es la métrica más adecuada para medir
la cercanía semántica entre textos, ya que compara la orientación de los
vectores y no su magnitud.

Además, cosine similarity es ampliamente utilizada en tareas de
búsqueda semántica y recuperación de información con embeddings densos,
como los generados por el modelo E5.

---

#### 2. ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?

El filtrado por metadata en **Qdrant** fue **mucho más sencillo** que en FAISS.

- En **FAISS**, no existe soporte nativo para metadata.
  Cualquier filtrado debe hacerse manualmente fuera del índice,
  lo que implica lógica adicional y mayor complejidad.

- En **Qdrant**, el uso de metadata (payload) es nativo.
  Se pueden aplicar filtros directamente sobre los campos de metadata
  durante la consulta (por ejemplo, `doc_id`, `chunk_id`, etiquetas, etc.)
  sin necesidad de procesar resultados manualmente.

Esto hace que Qdrant sea más adecuado para aplicaciones reales
donde se requiere combinar búsqueda vectorial con filtros estructurados.

---

#### 3. ¿Qué pasa con el tiempo de respuesta cuando aumentas k?

Al aumentar el valor de **k**, el tiempo de respuesta también aumenta,
ya que el sistema debe recuperar y ordenar un mayor número de resultados.

Sin embargo, en Qdrant este incremento es **gradual y controlado**,
gracias a sus estructuras de indexación optimizadas para búsqueda vectorial.

En general:
- Valores pequeños de k → respuestas muy rápidas
- Valores grandes de k → mayor costo computacional, pero aún eficiente

Esto permite ajustar k según las necesidades de precisión y rendimiento
de la aplicación.


## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo
Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Qué debes implementar
1. Conectar a Milvus.
2. Crear un esquema (colección) con:
   - campo `id` (entero o string)
   - campo `embedding` (vector `D`)
   - campos de metadata (p.ej., `category`, `source`, `title`)
3. Insertar `N` embeddings.
4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).
5. Ejecutar consultas Top-k y recuperar textos asociados.

### Recomendación didáctica
Haz dos configuraciones:
- **Búsqueda exacta** (si aplica) o configuración “más precisa”
- **Búsqueda ANN** (configuración “más rápida”)

Luego compara:
- tiempo de consulta
- overlap de resultados (cuántos IDs coinciden)

### Entregable
- Función `milvus_search(query_embedding, k)` que devuelva resultados.
- Un mini experimento: `k=5` y `k=20` (tiempos y resultados).

### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?


In [21]:
!pip -q install pymilvus milvus-lite

### 4.1 Conexión a Milvus (Milvus Lite)

Usaremos una base local `milvus_demo.db` (no requiere servidor).

In [22]:
from pymilvus import connections

MILVUS_URI = "milvus_demo.db"  # archivo local (Milvus Lite)
connections.connect(alias="default", uri=MILVUS_URI)

print("Conectado a Milvus Lite:", MILVUS_URI)

Conectado a Milvus Lite: milvus_demo.db


### 4.2 Preparar metadata requerida (category, source, title)

Milvus necesita que los campos de metadata existan en el esquema.
Aquí generamos valores simples y reproducibles.

In [23]:
# Asegurar que existen: embeddings (NxD), texts (N), metadatas (N dicts)

N = len(embeddings)
D = embeddings.shape[1]

ids = list(range(N))
category = ["wikipedia"] * N
source = ["wiki_corpus"] * N

# title basado en doc_id si existe, sino id
titles = []
for i in range(N):
    if isinstance(metadatas[i], dict) and "doc_id" in metadatas[i]:
        titles.append(f"doc_{metadatas[i]['doc_id']}")
    else:
        titles.append(f"doc_{i}")

print("Preparado:", N, "vectores | D =", D)


Preparado: 5000 vectores | D = 768


### 4.3 Crear esquema y colecciones

Crearemos dos colecciones:
- `docs_exact`: índice FLAT (más preciso)
- `docs_ann`: índice HNSW (más rápido)

In [24]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection, utility

def create_collection(name: str, dim: int):
    if utility.has_collection(name):
        utility.drop_collection(name)

    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dim),
        FieldSchema(name="category", dtype=DataType.VARCHAR, max_length=64),
        FieldSchema(name="source", dtype=DataType.VARCHAR, max_length=128),
        FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=128),
        FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=2048),  # preview del texto
    ]

    schema = CollectionSchema(fields, description=f"Collection {name}")
    col = Collection(name=name, schema=schema)
    return col

col_exact = create_collection("docs_exact", D)
col_ann   = create_collection("docs_ann", D)

print("Colecciones creadas:", col_exact.name, col_ann.name)

Colecciones creadas: docs_exact docs_ann


### 4.4 Insertar N embeddings

Guardaremos un `text` recortado (preview) para poder retornarlo fácil.

In [25]:
# Milvus espera listas nativas (no numpy)
vectors = embeddings.astype("float32").tolist()
text_preview = [t[:2000] for t in texts]  # max_length 2048

data = [
    ids,
    vectors,
    category,
    source,
    titles,
    text_preview
]

col_exact.insert(data)
col_ann.insert(data)

# Flush (asegura persistencia)
col_exact.flush()
col_ann.flush()

print("Insertados en ambas colecciones:", N)

Insertados en ambas colecciones: 5000


### 4.5 Crear índices

- Exacto: FLAT (búsqueda exacta)
- ANN: HNSW (rápido)

Luego cargamos las colecciones en memoria.

In [26]:
# Índice exacto (FLAT)
index_exact = {
    "index_type": "FLAT",
    "metric_type": "COSINE",
    "params": {}
}

# Índice ANN (IVF_FLAT) - aproximado y más rápido
index_ann = {
    "index_type": "IVF_FLAT",
    "metric_type": "COSINE",
    "params": {"nlist": 128}
}

# Crear índices
col_exact.create_index(field_name="embedding", index_params=index_exact)
col_ann.create_index(field_name="embedding", index_params=index_ann)

# Cargar colecciones
col_exact.load()
col_ann.load()

print("Índices creados (FLAT e IVF_FLAT) y colecciones cargadas")

Índices creados (FLAT e IVF_FLAT) y colecciones cargadas


### 4.6 Función entregable: milvus_search(query_embedding, k)

Permitimos elegir modo:
- mode="exact" (FLAT)
- mode="ann_fast" (HNSW rápido)
- mode="ann_precise" (HNSW más preciso)


In [27]:
import time

def milvus_search(query_embedding, k=5, mode="ann_fast"):
    """
    mode:
      - exact: FLAT (preciso)
      - ann_fast: IVF_FLAT nprobe bajo (rápido)
      - ann_precise: IVF_FLAT nprobe alto (más preciso)
    """
    q = query_embedding[0].astype("float32").tolist()

    if mode == "exact":
        col = col_exact
        search_params = {"metric_type": "COSINE", "params": {}}
    elif mode == "ann_precise":
        col = col_ann
        search_params = {"metric_type": "COSINE", "params": {"nprobe": 32}}
    else:  # ann_fast
        col = col_ann
        search_params = {"metric_type": "COSINE", "params": {"nprobe": 8}}

    t0 = time.perf_counter()
    res = col.search(
        data=[q],
        anns_field="embedding",
        param=search_params,
        limit=k,
        output_fields=["text", "category", "source", "title"]
    )
    t_ms = (time.perf_counter() - t0) * 1000

    out = []
    for hit in res[0]:
        payload = {
            "category": hit.entity.get("category"),
            "source": hit.entity.get("source"),
            "title": hit.entity.get("title"),
        }
        out.append((hit.id, float(hit.score), hit.entity.get("text"), payload))

    return out, t_ms

## Mini experimento: k=5 y k=20 (tiempos + overlap)

Compararemos:
- exact (FLAT)
- ann_fast (HNSW ef=32)
- ann_precise (HNSW ef=128)

Medimos:
- tiempo en ms
- overlap de IDs (cuántos resultados coinciden con exact)


In [28]:
import pandas as pd

query_text = "Battery measuring"
query_embedding = embed_query(query_text)

def run_experiment(k):
    exact_res, exact_t = milvus_search(query_embedding, k=k, mode="exact")
    fast_res, fast_t   = milvus_search(query_embedding, k=k, mode="ann_fast")
    prec_res, prec_t   = milvus_search(query_embedding, k=k, mode="ann_precise")

    exact_ids = {r[0] for r in exact_res}
    fast_ids  = {r[0] for r in fast_res}
    prec_ids  = {r[0] for r in prec_res}

    return pd.DataFrame([
        {"mode": "exact (FLAT)", "k": k, "time_ms": exact_t, "overlap_with_exact": k},
        {"mode": "ann_fast (IVF_FLAT nprobe=8)", "k": k, "time_ms": fast_t, "overlap_with_exact": len(exact_ids & fast_ids)},
        {"mode": "ann_precise (IVF_FLAT nprobe=32)", "k": k, "time_ms": prec_t, "overlap_with_exact": len(exact_ids & prec_ids)},
    ])

df_results = pd.concat([run_experiment(5), run_experiment(20)], ignore_index=True)
df_results

,mode,k,time_ms,overlap_with_exact
0,exact (FLAT),5,6.431390,5
1,ann_fast (IVF_FLAT nprobe=8),5,4.408953,5
2,ann_precise (IVF_FLAT nprobe=32),5,3.278026,5
3,exact (FLAT),20,5.177107,20
4,ann_fast (IVF_FLAT nprobe=8),20,3.888618,20
5,ann_precise (IVF_FLAT nprobe=32),20,3.232040,20


### Respuestas – Parte 4: Milvus (ANN y escalabilidad)

#### 1. ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?

En Milvus Lite se utilizó el índice **IVF_FLAT**, y el principal parámetro ajustado
para controlar el equilibrio entre **precisión y velocidad** fue **`nprobe`** durante la búsqueda.

- **`nlist`** (parámetro del índice):
  - Define en cuántas particiones se divide el espacio vectorial.
  - Un `nlist` mayor mejora la organización del índice, pero incrementa el costo de construcción.

- **`nprobe`** (parámetro de búsqueda):
  - Controla cuántas particiones (`nlist`) se exploran durante la consulta.
  - `nprobe` bajo (ej. 8):  
    → búsqueda más rápida, menor precisión.
  - `nprobe` alto (ej. 32):  
    → búsqueda más precisa, mayor tiempo de respuesta.

De esta forma:
- **FLAT** representa la búsqueda exacta (máxima precisión).
- **IVF_FLAT + nprobe bajo** representa ANN rápido.
- **IVF_FLAT + nprobe alto** representa ANN más preciso.

---

#### 2. ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?

La evidencia se obtuvo comparando los resultados de **ANN (IVF_FLAT)** contra
la búsqueda **exacta (FLAT)** mediante el **overlap de IDs** recuperados.

Se observó que:
- Cuando `nprobe` es bajo, el número de IDs coincidentes con la búsqueda exacta
  es menor que `k`, lo que indica que ANN no siempre recupera los mismos vecinos
  más cercanos.
- Al aumentar `nprobe`, el overlap aumenta y los resultados se aproximan más
  a la búsqueda exacta, aunque con mayor costo computacional.

Esto demuestra que ANN introduce una pequeña variación en los resultados,
aceptando una leve pérdida de exactitud a cambio de una mejora en velocidad,
lo cual es el comportamiento esperado y deseado en sistemas escalables.


## Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

### Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

### Qué debes implementar
1. Conectar a Weaviate.
2. Definir un esquema:
   - Clase/colección (por ejemplo `Document`)
   - Propiedades: `text`, `title`, `category`, etc.
   - Vector asociado (embedding)
3. Insertar objetos con:
   - propiedades + vector
4. Consultar por similitud (Top-k) con `query_embedding`.
5. (Opcional) agregar un filtro por propiedad (metadata).

### Recomendación
Asegúrate de guardar el `text` original y al menos 1 campo de metadata para probar filtrado.

### Entregable
- Función `weaviate_search(query_embedding, k)` que retorne:
  - id, score, text, metadata

### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
- ¿Cómo describirías el trade-off de complejidad vs expresividad?


### Instalación de Weaviate (cliente + modo embebido)

In [29]:
!pip uninstall -y weaviate-client
!pip install -q "weaviate-client>=3.26.7,<4.0.0"

Found existing installation: weaviate-client 3.26.7
Uninstalling weaviate-client-3.26.7:
  Successfully uninstalled weaviate-client-3.26.7


### 5.1 Conectar a Weaviate (modo local / embedded)

No usamos Docker ni servidor externo.

In [30]:
import weaviate

client = weaviate.Client(
    embedded_options=weaviate.embedded.EmbeddedOptions()
)

print("Conectado a Weaviate (embedded, v3)")

Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.0/weaviate-v1.23.0-Linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 21651
Conectado a Weaviate (embedded, v3)


### 5.2 Definir esquema (clase Document)

La clase tendrá:
- text (string)
- title (string)
- category (string)
- vector (embedding manual)

In [31]:
# Borrar esquema previo si existe
if client.schema.exists("Document"):
    client.schema.delete_class("Document")

schema = {
    "class": "Document",
    "description": "Documentos con embeddings semánticos",
    "vectorizer": "none",  # usamos embeddings manuales
    "properties": [
        {"name": "text", "dataType": ["text"]},
        {"name": "title", "dataType": ["string"]},
        {"name": "category", "dataType": ["string"]},
    ]
}

client.schema.create_class(schema)

print("Esquema creado: Document")

Esquema creado: Document


### 5.3 Insertar objetos (propiedades + vector)

Cada objeto tendrá:
- propiedades (text, title, category)
- vector (embedding)

In [32]:
from tqdm.auto import tqdm

client.batch.configure(batch_size=64)

with client.batch as batch:
    for i in tqdm(range(len(embeddings))):
        properties = {
            "text": texts[i],
            "title": f"doc_{metadatas[i].get('doc_id', i)}",
            "category": "wikipedia"
        }
        batch.add_data_object(
            data_object=properties,
            class_name="Document",
            vector=embeddings[i].tolist()
        )

print("Objetos insertados en Weaviate:", len(embeddings))

  0%|          | 0/5000 [00:00<?, ?it/s]

Objetos insertados en Weaviate: 5000


### 5.4 Función entregable: weaviate_search(query_embedding, k)

Retorna:
- id
- score (distance)
- text
- metadata

In [33]:
def weaviate_search(query_embedding, k=5):
    res = (
        client.query
        .get("Document", ["text", "title", "category"])
        .with_near_vector({
            "vector": query_embedding[0].tolist()
        })
        .with_limit(k)
        .with_additional(["distance", "id"])
        .do()
    )

    out = []
    for r in res["data"]["Get"]["Document"]:
        out.append((
            r["_additional"]["id"],
            r["_additional"]["distance"],
            r["text"][:220] + ("..." if len(r["text"]) > 220 else ""),
            {
                "title": r["title"],
                "category": r["category"]
            }
        ))
    return out

### Ejemplo de búsqueda Top-k

In [34]:
query_text = "Battery measuring"
query_embedding = embed_query(query_text)

weaviate_search(query_embedding, k=5)

[('b4ef6bd5-04ca-4da5-91d5-dde198fb3a4d',
  0.1381998,
  "Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It is particularly imp...",
  {'title': 'doc_1', 'category': 'wikipedia'}),
 ('3aa36004-451a-4561-ae84-6799d2bb6eba',
  0.16837788,
  'otective diodes cannot be used, a battery will simply destroy the diodes and damage itself. An ESR meter known not to have diode protection will give a reading of internal resistance for a rechargeable or non-rechargeabl...',
  {'title': 'doc_1', 'category': 'wikipedia'}),
 ('17c33735-8f94-4aa1-92f3-e8b6c839e139',
  0.1777972,
  "ad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively, an ammeter may be fitted. This indicates whether the battery is being charged or discharged. In the adjacent picture, t...",
  {'title': 'doc_1', 'category': 'wikipedia'}),
 ('

### (Opcional) Búsqueda con filtro por metadata

Ejemplo: category = "wikipedia"

In [35]:
def weaviate_search_filtered(query_embedding, k=5):
    res = (
        client.query
        .get("Document", ["text", "title", "category"])
        .with_near_vector({
            "vector": query_embedding[0].tolist()
        })
        .with_where({
            "path": ["category"],
            "operator": "Equal",
            "valueString": "wikipedia"
        })
        .with_limit(k)
        .with_additional(["distance", "id"])
        .do()
    )

    return res["data"]["Get"]["Document"]

weaviate_search_filtered(query_embedding, k=5)

[{'_additional': {'distance': 0.1381998,
   'id': 'b4ef6bd5-04ca-4da5-91d5-dde198fb3a4d'},
  'category': 'wikipedia',
  'text': "Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It is particularly important in the case of a battery electric vehicle. Some automobiles are fitted with a battery condition meter to monitor the starter battery. This meter is, essentially, a voltmeter but it may also be marked with coloured zones for easy visualization. Many newer cars no longer offer voltmeters or ammeters; instead, these vehicles typically have a light with the outline of an automotive battery on it. This can be somewhat misleading as it may be confused for an indicator of a bad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively,",
  'title': 'doc_1'},
 {'_additional': {'distance': 0.16837788,
   'i

### Preguntas – Parte 5: Weaviate

#### 1. ¿Qué diferencia conceptual hay entre “schema + objetos” y “tabla + filas”?

En Weaviate, el modelo **schema + objetos** es semántico:
- El esquema define **clases** con significado (por ejemplo, Document).
- Los objetos representan entidades del mundo real con propiedades y vectores.
- El vector es parte integral del objeto.

En contraste, el modelo **tabla + filas** (bases relacionales):
- Es estructural y rígido.
- Las filas no tienen semántica más allá de sus columnas.
- No existe noción nativa de similitud o embeddings.

Weaviate se asemeja más a un **modelo orientado a conocimiento** que a un modelo tabular.

---

#### 2. ¿Cómo describirías el trade-off entre complejidad y expresividad?

Weaviate es **más expresivo**, pero también **más complejo**:

- Ventajas:
  - Esquemas ricos y semánticos.
  - Metadata integrada al modelo.
  - Búsqueda vectorial + filtros declarativos.
  - Más cercano a sistemas de conocimiento reales.

- Desventajas:
  - Mayor complejidad conceptual.
  - Curva de aprendizaje más alta que FAISS o Milvus básico.


## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

### Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Qué debes implementar
1. Crear una colección.
2. Insertar:
   - ids
   - embeddings
   - documents (texto)
   - metadatas (opcional)
3. Consultar Top-k con `query_embedding`.

### Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

### Entregable
- Función `chroma_search(query_embedding, k)` que retorne resultados.
- Una consulta con `k=5`.

### Preguntas
- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
- ¿Qué limitaciones ves para un sistema en producción?


### Instalación de ChromaDB


In [36]:
!pip -q install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.9 MB/s eta 

In [37]:
import chromadb
print("Chroma version:", chromadb.__version__)

Chroma version: 1.4.0


### 6.1 Crear cliente y colección

Usaremos un cliente simple (sin servidor).

In [38]:
import chromadb

chroma_client = chromadb.Client()

COLLECTION_NAME = "documents_chroma"
collection = chroma_client.get_or_create_collection(name=COLLECTION_NAME)

print("Colección creada/lista:", COLLECTION_NAME)

Colección creada/lista: documents_chroma


### 6.2 Preparar datos: ids, embeddings, documents, metadatas

In [40]:
# ids deben ser strings en Chroma
ids_chroma = [str(i) for i in range(len(embeddings))]

documents = texts  # texto original
embeddings_list = embeddings.astype("float32").tolist()

# metadatas opcionales (usa los que ya tienes)
metadatas_chroma = []
for i in range(len(embeddings)):
    md = metadatas[i] if isinstance(metadatas[i], dict) else {}
    # agregamos campos extra útiles
    md = {**md, "source": "wikipedia"}
    metadatas_chroma.append(md)

print("Preparado:")
print("ids:", len(ids_chroma))
print("documents:", len(documents))
print("embeddings:", len(embeddings_list))
print("metadatas:", len(metadatas_chroma))

Preparado:
ids: 5000
documents: 5000
embeddings: 5000
metadatas: 5000


### 6.3 Insertar en Chroma

Para evitar duplicados si se re-ejecuta la celda, primero intentamos limpiar la colección.

In [41]:
# Limpiar colección si ya tenía datos
try:
    chroma_client.delete_collection(COLLECTION_NAME)
    collection = chroma_client.get_or_create_collection(name=COLLECTION_NAME)
except Exception:
    pass

collection.add(
    ids=ids_chroma,
    embeddings=embeddings_list,
    documents=documents,
    metadatas=metadatas_chroma
)

print("Insertado en Chroma:", len(ids_chroma))

Insertado en Chroma: 5000


## Entregable: chroma_search(query_embedding, k)

Retornará una lista con:
- id
- score (distancia; menor suele ser mejor dependiendo configuración interna)
- text
- metadata

In [44]:
def chroma_search(query_embedding, k=5):
    res = collection.query(
        query_embeddings=query_embedding.astype("float32").tolist(),
        n_results=k,
        include=["documents", "metadatas", "distances"]
    )

    out = []
    for i in range(min(k, len(res["ids"][0]))):
        out.append((
            res["ids"][0][i],
            float(res["distances"][0][i]),
            res["documents"][0][i][:220] + ("..." if len(res["documents"][0][i]) > 220 else ""),
            res["metadatas"][0][i]
        ))
    return out

### Ejemplo de consulta con k=5

In [45]:
query_text = "Battery measuring"
query_embedding = embed_query(query_text)

chroma_search(query_embedding, k=5)

[('1',
  0.27639931440353394,
  "Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It is particularly imp...",
  {'source': 'wikipedia', 'chunk_id': 0, 'doc_id': 1}),
 ('5',
  0.33675602078437805,
  'otective diodes cannot be used, a battery will simply destroy the diodes and damage itself. An ESR meter known not to have diode protection will give a reading of internal resistance for a rechargeable or non-rechargeabl...',
  {'source': 'wikipedia', 'doc_id': 1, 'chunk_id': 4}),
 ('2',
  0.35559433698654175,
  "ad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively, an ammeter may be fitted. This indicates whether the battery is being charged or discharged. In the adjacent picture, t...",
  {'chunk_id': 1, 'source': 'wikipedia', 'doc_id': 1}),
 ('14',
  0.3652741312980652,
  'Capacity loss Capacity

### Preguntas – Parte 6: Chroma

#### 1) ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
Fue más fácil y rápido con Chroma porque:
- No requiere levantar servidor ni configurar colecciones complejas.
- La API es directa: `add()` para insertar y `query()` para buscar.
- Es ideal para prototipos en notebooks y pruebas rápidas.

En comparación:
- Qdrant y Milvus requieren más configuración (colecciones/esquemas/índices),
  pero ofrecen más control y características para producción.

---

#### 2) ¿Qué limitaciones ves para un sistema en producción?
Limitaciones típicas de Chroma en producción:
- Menor enfoque en despliegue distribuido y alta concurrencia.
- Menos opciones avanzadas de indexación y tuning comparado con Milvus.
- Menos robustez/observabilidad (monitoring, escalado, administración) que soluciones orientadas a producción.
- Para sistemas grandes (millones de vectores), suele ser más adecuado Milvus/Qdrant/Weaviate
  por su enfoque en rendimiento, escalabilidad y operación.


## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Qué debes implementar
1. Conectar a una base PostgreSQL con `pgvector` habilitado.
2. Crear una tabla (ej. `documents`) con:
   - `id` (PK)
   - `text` (texto)
   - `embedding` (vector(D))
   - metadata (columnas adicionales)
3. Insertar todos los documentos y embeddings.
4. Consultar Top-k por similitud, ordenando por distancia.

### Fórmula conceptual (lo que implementa tu SQL)
Para una consulta `q`, buscas:
$$ argmin_d \in D \; \text{dist}(\vec{q}, \vec{d})$$
donde `dist` puede ser L2 o una variante para cosine (según configuración).

### Entregable
- Función `pgvector_search(query_embedding, k)` que ejecute SQL y devuelva:
  - id, score/distancia, text, metadata

### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?
- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?


## 7.1 Instalar PostgreSQL y pgvector


In [48]:
!apt-get -qq update
!apt-get -qq install -y postgresql postgresql-contrib postgresql-common

# Intentar instalar pgvector desde repos
!apt-get -qq install -y postgresql-pgvector || echo "pgvector no está en apt. Usaremos instalación manual."


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Preconfiguring packages ...
Selecting previously unselected package logrotate.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../00-logrotate_3.19.0-1ubuntu1.1_amd64.deb ...
Unpacking logrotate (3.19.0-1ubuntu1.1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../01-netbase_6.3_all.deb ...
Unpacking netbase (6.3) ...
Selecting previously unselected package libcommon-sense-perl:amd64.
Preparing to unpack .../02-libcommon-sense-perl_3.75-2build1_amd64.deb ...
Unpacking libcommon-sense-perl:amd64 (3.75-2build1) ...
Selecting previously unselected package libjson-perl.
Preparing to unpack .../03-libjson-perl_4.04000-1_all.deb ...
Unpacking libjson-perl (4.04000-1) ...
Selecting previously unselected package libtypes-serialiser-perl

### 7.1.b Instalar pgvector manualmente desde GitHub


In [49]:
import os, subprocess, textwrap, sys, re, pathlib, json, math

In [50]:
# Instalación manual de pgvector si no existe el paquete de apt
# Esto compila la extensión para el PostgreSQL instalado.

!apt-get -qq install -y git build-essential postgresql-server-dev-all

!rm -rf /tmp/pgvector
!git clone -q https://github.com/pgvector/pgvector.git /tmp/pgvector
!cd /tmp/pgvector && make -q
!cd /tmp/pgvector && make -q install

print("pgvector compilado e instalado")

Preconfiguring packages ...
(Reading database ... 123660 files and directories currently installed.)
Preparing to unpack .../libc6-dev_2.35-0ubuntu3.11_amd64.deb ...
Unpacking libc6-dev:amd64 (2.35-0ubuntu3.11) over (2.35-0ubuntu3.8) ...
Preparing to unpack .../libc-dev-bin_2.35-0ubuntu3.11_amd64.deb ...
Unpacking libc-dev-bin (2.35-0ubuntu3.11) over (2.35-0ubuntu3.8) ...
Preparing to unpack .../libc6_2.35-0ubuntu3.11_amd64.deb ...
Unpacking libc6:amd64 (2.35-0ubuntu3.11) over (2.35-0ubuntu3.8) ...
Setting up libc6:amd64 (2.35-0ubuntu3.11) ...
Selecting previously unselected package python3-yaml.
(Reading database ... 123660 files and directories currently installed.)
Preparing to unpack .../00-python3-yaml_5.4.1-1ubuntu1_amd64.deb ...
Unpacking python3-yaml (5.4.1-1ubuntu1) ...
Selecting previously unselected package binfmt-support.
Preparing to unpack .../01-binfmt-support_2.2.1-2_amd64.deb ...
Unpacking binfmt-support (2.2.1-2) ...
Selecting previously unselected package libclang-cp

## 7.2 Iniciar PostgreSQL y crear usuario/base de datos

Crearemos:
- usuario: `nelson`
- password: `nelson123`
- database: `vectordb`

In [55]:
# Iniciar PostgreSQL
!service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


In [56]:
# Crear usuario nelson (si ya existe, mostrará aviso y seguimos)
!sudo -u postgres psql -c "CREATE USER nelson WITH PASSWORD 'nelson123';" || echo "Usuario ya existe"

CREATE ROLE


In [57]:
# Crear base de datos vectordb
!sudo -u postgres psql -c "CREATE DATABASE vectordb OWNER nelson;" || echo "Base de datos ya existe"

CREATE DATABASE


In [58]:
!sudo -u postgres psql -c "ALTER DATABASE vectordb OWNER TO nelson;"
!sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE vectordb TO nelson;"

ALTER DATABASE
GRANT


In [65]:
!sudo -u postgres psql -c "\pset pager off" -c "\du"
!sudo -u postgres psql -c "\pset pager off" -c "\l"

Pager usage is off.
                                   List of roles
 Role name |                         Attributes                         | Member of 
-----------+------------------------------------------------------------+-----------
 nelson    |                                                            | {}
 postgres  | Superuser, Create role, Create DB, Replication, Bypass RLS | {}

Pager usage is off.
                                  List of databases
   Name    |  Owner   | Encoding |   Collate   |    Ctype    |   Access privileges   
-----------+----------+----------+-------------+-------------+-----------------------
 postgres  | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | 
 template0 | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | =c/postgres          +
           |          |          |             |             | postgres=CTc/postgres
 template1 | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | =c/postgres          +
           |          |          |       

## 7.3 Conexión desde Python a PostgreSQL local (localhost)


In [105]:
import psycopg2
from pgvector.psycopg2 import register_vector

DB_CONFIG = {
    "host": "127.0.0.1",
    "port": 5432,
    "database": "vectordb",
    "user": "nelson",
    "password": "nelson123"
}

conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

# Asegurar que la extensión exista (idempotente)
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
conn.commit()

# Registrar tipo vector para psycopg2
register_vector(conn)

print("Conectado a PostgreSQL local y pgvector disponible")


Conectado a PostgreSQL local y pgvector disponible


In [106]:
D = embeddings.shape[1]

cur.execute("DROP TABLE IF EXISTS documents;")
cur.execute(f"""
CREATE TABLE documents (
    id INTEGER PRIMARY KEY,
    text TEXT,
    embedding VECTOR({D}),
    doc_id INTEGER,
    chunk_id INTEGER,
    source TEXT
);
""")
conn.commit()

print(f"Tabla documents creada (D={D})")

Tabla documents creada (D=768)


In [107]:
insert_sql = """
INSERT INTO documents (id, text, embedding, doc_id, chunk_id, source)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for i in range(len(embeddings)):
    md = metadatas[i] if isinstance(metadatas[i], dict) else {}
    cur.execute(
        insert_sql,
        (
            int(i),
            texts[i],
            embeddings[i].tolist(),
            int(md.get("doc_id", -1)),
            int(md.get("chunk_id", -1)),
            "wikipedia"
        )
    )

conn.commit()
print("Insertados:", len(embeddings))

Insertados: 5000


In [114]:
def pgvector_search(query_embedding, k=5):
    sql = """
    SELECT
        id,
        (embedding <=> %s::vector) AS distance,
        text,
        doc_id,
        chunk_id,
        source
    FROM documents
    ORDER BY embedding <=> %s::vector
    LIMIT %s;
    """
    q = query_embedding[0].tolist()
    try:
        cur.execute(sql, (q, q, k))
        rows = cur.fetchall()
    except Exception as e:
        conn.rollback()
        raise e

    results = []
    for r in rows:
        results.append((r[0], float(r[1]), r[2], {"doc_id": r[3], "chunk_id": r[4], "source": r[5]}))
    return results

In [115]:
query_text = "Battery measuring"
query_embedding = embed_query(query_text)

pgvector_search(query_embedding, k=5)

[(1,
  0.13819967052940485,
  "Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It is particularly important in the case of a battery electric vehicle. Some automobiles are fitted with a battery condition meter to monitor the starter battery. This meter is, essentially, a voltmeter but it may also be marked with coloured zones for easy visualization. Many newer cars no longer offer voltmeters or ammeters; instead, these vehicles typically have a light with the outline of an automotive battery on it. This can be somewhat misleading as it may be confused for an indicator of a bad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively,",
  {'doc_id': 1, 'chunk_id': 0, 'source': 'wikipedia'}),
 (5,
  0.16837804505955756,
  'otective diodes cannot be used, a battery will simply destroy the diodes and da

## ¿Qué tan “explicable” te parece esta aproximación vs las otras?

La aproximación con **PostgreSQL + pgvector** es más explicable que las bases vectoriales dedicadas, ya que la búsqueda se define explícitamente mediante una consulta SQL. Esto permite ver claramente cómo se calcula la distancia, cómo se ordenan los resultados y qué información se recupera.

En comparación, motores como FAISS, Milvus o Qdrant abstraen gran parte de la lógica interna, lo que mejora el rendimiento pero reduce la transparencia del proceso.

---

## ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?

El uso de SQL permite combinar la búsqueda vectorial con datos relacionales mediante `JOIN`, lo que facilita integrar embeddings con tablas de metadata, usuarios o categorías. Esto resulta muy útil en sistemas híbridos donde conviven datos estructurados y semánticos.

Además, SQL permite aplicar filtros complejos y realizar agregaciones o análisis estadísticos de forma nativa, aprovechando un ecosistema robusto y ampliamente conocido.

---

## ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?

Aunque pgvector es una solución flexible, presenta limitaciones de escalabilidad frente a bases vectoriales dedicadas. PostgreSQL no está diseñado para manejar millones de vectores ni consultas vectoriales intensivas de manera continua.

Por ello, pgvector es más adecuado para proyectos pequeños o medianos, mientras que soluciones como Milvus o FAISS son preferibles en escenarios de gran escala y alta concurrencia.